In [4]:
import os
import json
import pandas as pd
import numpy as np

In [9]:
CATEGORY = "설명글"  # 글짓기, 설명글
CLASS = "중등_2학년"
TOPIC = "본인의 셩격"
# 나의 위인전
# 혐오시설 건설문제에 대한 본인의 생각
# 영화/독서감상문
FILE_PATH = f'preprocess_data/{TOPIC}'


In [10]:
CONT0 = "주제의 명료성"
CONT1 = "설명의 구체성"
CONT2 = "프롬프트 독해력"  # 확정
CONT3 = "사고의 창의성"
EXP0 = "문법의 정확성"
EXP1 = "단어 사용의 적절성"
EXP2 = "문장 표현의 적절성"  # 확정
ORG0 = "문단 간 구조의 적절성"  # 확정
ORG1 = "문단 내 구조의 적절성"
ORG2 = "구조의 일관성"  # 확정
ORG3 = "분량의 적절성"

In [11]:
train_directory = './original_data/01.데이터/1.Training/라벨링데이터/' + CATEGORY

# Create an empty DataFrame to store the results
train_df = pd.DataFrame(
    columns=['text', CONT0, CONT1, CONT2, CONT3, EXP0, EXP1, EXP2, ORG0, ORG1, ORG2, ORG3])

for filename in os.listdir(train_directory):
    # Check if the filename starts with "Writing_high school_1st grade_ESSAY"
    if filename.startswith(f"{CATEGORY}_{CLASS}_ESSAY"):
        # Open the file
        with open(os.path.join(train_directory, filename), 'r', encoding='utf-8') as f:
            # Load the JSON data
            data = json.load(f)

            topic = data["rubric"]["essay_main_subject"]
            if topic == TOPIC:
                # Get the data from 'essay_scoreT_org'
                text = ""

                for paragraph in data['paragraph']:
                    text += paragraph['paragraph_txt'].replace('#@문장구분#', '')

                cont_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_cont'])
                org_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_org'])
                exp_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_exp'])

                # Compute the averages and round them
                cont_avg = np.round(cont_scores.mean(axis=0))
                exp_avg = np.round(exp_scores.mean(axis=0))
                org_avg = np.round(org_scores.mean(axis=0))

                # Add to DataFrame
                train_df = train_df.append({
                    'text': text,
                    CONT0: cont_avg[0],
                    CONT1: cont_avg[1],
                    CONT2: cont_avg[2],
                    CONT3: cont_avg[3],
                    EXP0: exp_avg[0],
                    EXP1: exp_avg[1],
                    EXP2: exp_avg[2],
                    ORG0: org_avg[0],
                    ORG1: org_avg[1],
                    ORG2: org_avg[2],
                    ORG3: org_avg[3]
                }, ignore_index=True)

train_df.to_excel(f'{FILE_PATH}_train.xlsx', index=False)

In [12]:
# train_directory = './original_data/01.데이터/1.Training/라벨링데이터/글짓기'
val_directory = './original_data/01.데이터/2.Validation/라벨링데이터/' + CATEGORY

# Create an empty DataFrame to store the results
val_df = pd.DataFrame(
    columns=['text', CONT0, CONT1, CONT2, CONT3, EXP0, EXP1, EXP2, ORG0, ORG1, ORG2, ORG3])

# Scan through the directory
for filename in os.listdir(val_directory):
    # Check if the filename starts with "Writing_high school_1st grade_ESSAY"
    if filename.startswith(f"{CATEGORY}_{CLASS}_ESSAY"):
        # Open the file
        with open(os.path.join(val_directory, filename), 'r', encoding='utf-8') as f:
            # Load the JSON data
            data = json.load(f)

            topic = data["rubric"]["essay_main_subject"]

            if topic == TOPIC:
                # Get the data from 'essay_scoreT_org'
                text = ""

                for paragraph in data['paragraph']:
                    text += paragraph['paragraph_txt'].replace('#@문장구분#', '')

                cont_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_cont'])
                org_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_org'])
                exp_scores = np.array(data['score']['essay_scoreT_detail']['essay_scoreT_exp'])

                # Compute the averages and round them
                cont_avg = np.round(cont_scores.mean(axis=0))
                org_avg = np.round(org_scores.mean(axis=0))
                exp_avg = np.round(exp_scores.mean(axis=0))

                # Add to DataFrame
                val_df = val_df.append({
                    'text': text,
                    CONT0: cont_avg[0],
                    CONT1: cont_avg[1],
                    CONT2: cont_avg[2],
                    CONT3: cont_avg[3],
                    EXP0: exp_avg[0],
                    EXP1: exp_avg[1],
                    EXP2: exp_avg[2],
                    ORG0: org_avg[0],
                    ORG1: org_avg[1],
                    ORG2: org_avg[2],
                    ORG3: org_avg[3]
                }, ignore_index=True)

# Save DataFrame to Excel
val_df.to_excel(f'{FILE_PATH}_val.xlsx', index=False)